
# 📝 SportsCrew · **Ollama (ejercicio para alumnos)**
Completa las celdas marcadas con **TODO**. Hay **pistas** en los comentarios de cada celda.

**Objetivo**: generar un *artículo breve* y **5 tweets** sobre una liga deportiva usando **Ollama (mistral)** con **CrewAI**.

**Documentación útil**
- **Ollama (oficial):** https://github.com/ollama/ollama  
- **CrewAI (conceptos):** https://docs.crewai.com/concepts/  
- **CrewAI (API referencia):** https://docs.crewai.com/reference/  
- **LiteLLM (enrutador):** https://docs.litellm.ai/  
- **LangChain (ecosistema):** https://python.langchain.com/docs/  
- **DuckDuckGo (herramienta LC):** https://python.langchain.com/docs/integrations/tools/ddg  
- **Requests (HTTP):** https://requests.readthedocs.io/  
- **python-dotenv:** https://github.com/theskumar/python-dotenv  
- **json (stdlib):** https://docs.python.org/3/library/json.html




## 1) Instalar dependencias en este kernel
Rellena la lista `packages` con las librerías necesarias y completa la llamada a `pip install`.

**Pistas**:
- Necesitarás: `crewai`, `litellm`, `langchain`, `langchain-community`, `langchain-openai`, `duckduckgo-search`, `requests`, `python-dotenv`.
- Usa `sys.executable -m pip install -U ...` para asegurar instalación en este kernel.


In [ ]:
import importlib, sys, subprocess
packages = [
    "crewai>=0.63.6",
    "litellm>=1.40.11",
    "langchain>=0.2.10",
    "langchain-community>=0.2.10",
    "langchain-openai",
    "duckduckgo-search",
    "requests",
    "python-dotenv"
]
to_install = []
for spec in packages:
    mod = spec.split("==")[0].split(">=")[0].replace("-", "_")
    try:
        importlib.import_module(mod if mod!="python_dotenv" else "dotenv")
    except Exception:
        to_install.append(spec)
if to_install:
    print("Instalando:", to_install)
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", *to_install])
else:
    print("Todo estaba instalado ✅")



## 2) Comprobar **Ollama** y el modelo `mistral`
Debes verificar que:
1) El binario `ollama` está en el PATH.
2) La API local responde en `http://localhost:11434/api/tags`.
3) El modelo `mistral` está descargado.

**Pistas**:
- Usa `shutil.which("ollama")` para comprobar el binario.
- Usa `requests.get(url, timeout=5)` y revisa `resp.ok`.
- La respuesta JSON tiene una clave `models` con nombres; busca `"mistral"`.


In [ ]:


import shutil, requests
print("Binario 'ollama' en PATH:", "✅" if shutil.which("ollama") else "❌")
try:
    r = requests.get("http://localhost:11434/api/tags", timeout=5)
    if r.ok:
        tags = [m.get("name") for m in r.json().get("models", []) if isinstance(m, dict)]
        print("Ollama API:", "✅ OK")
        print("Modelos locales:", tags)
        print("'mistral' presente:", "✅" if any("mistral" in (t or "") for t in tags) else "❌ (ejecuta: ollama pull mistral)")
    else:
        print("Ollama API respondió con error HTTP:", r.status_code)
except Exception as e:
    print("No se pudo conectar a http://localhost:11434. ¿Ejecutaste 'ollama serve'?")
    print("Detalle:", e)




## 3) Elegir deporte y liga
Cambia estos valores si quieres otra competición. Ejemplos:
1. **soccer**: 
- `esp.1` (LaLiga)
- `eng.1` (Premier)
- `ita.1` (Calcio)
- `fra.1` (ligue 1) 
- `usa.1` (MLS)
2. **basketball**: 
- `nba`
- `wnba`
3. **football**: 
- `nfl` 
4. **baseball**: 
- `mlb` 
5. **hockey**: 
- `nhl`



In [ ]:

# TODO 3.1: elige deporte y liga
SPORT = ""   # p.ej., "soccer"
LEAGUE = ""  # p.ej., "esp.1"
MAX_ARTICLES = 5

print("Deporte:", SPORT, "| Liga:", LEAGUE, "| Máx artículos:", MAX_ARTICLES)



## 4) Descargar noticias (ESPN)
Construye la URL y parsea el JSON. Guarda los artículos en `articles` (máximo `MAX_ARTICLES`).

**Pistas**:
- URL base: `http://site.api.espn.com/apis/site/v2/sports/{sport}/{league}/news`
- A veces los datos están en `data["articles"]`; otras en `data["feed"]["entries"]`.
- Para `entries`, normaliza a `{"headline": ...}`.


In [ ]:

import requests

def get_news_url(sport, league):
    """
    Genera la URL para obtener noticias de ESPN según el deporte y la liga.

    Args:
        sport (str): Deporte (por ejemplo, "soccer").
        league (str): Liga específica (por ejemplo, "esp.1" para LaLiga).

    Returns:
        str: URL para obtener las noticias.
    """
    return f"http://site.api.espn.com/apis/site/v2/sports/{sport}/{league}/news"

# TODO 4.1: construye la URL
# url = ...

# TODO 4.2: GET y .json()
# resp = ...
# data = ...

# TODO 4.3: extrae lista de artículos; normaliza si vienen en 'feed'/'entries'
articles = []
# if isinstance(data, dict):
#     if ...:
#         articles = data["articles"]
#     elif ...:
#         entries = data["feed"].get("entries", [])
#         articles = [{"headline": e.get("headline") or e.get("title", "")} for e in entries]

print(f"Artículos encontrados: {len(articles)}")
articles = articles[:MAX_ARTICLES]



## 5) Búsqueda rápida (opcional) con DuckDuckGo
Intenta crear un wrapper y un `run` de búsqueda para obtener contexto por titular.

In [ ]:

ddg = None
try:
    # TODO 5.1: importa las clases requeridas
    # from ...
    # wrapper = DuckDuckGoSearchAPIWrapper(region="us-en", time="d")
    # ddg = DuckDuckGoSearchRun(api_wrapper=wrapper)
    pass
except Exception as e:
    print("DuckDuckGo no disponible. Detalle:", e)



## 6) Configurar el modelo (Ollama · mistral) para CrewAI
Crea un `LLM` de CrewAI que apunte a Ollama.


In [ ]:

from crewai import LLM

# TODO 6.1: crea la instancia de LLM para Ollama/mistral
llm = None  # reemplaza por LLM(...)

print("LLM configurado:", "✅" if llm else "❌")



## 7) Resúmenes (120–180 palabras)
Para cada artículo, crea un agente **Investigador** y genera un resumen.

**Pistas**:
- Importa: `from crewai import Agent, Task, Crew`
- `headline = art.get("headline") or art.get("title") or ""`
- Si `ddg` existe y hay `headline`, llama a `ddg.run(headline)` (maneja excepciones).
- Prompt: “Resume en 120–180 palabras. SOLO el resumen...” (añade titular y contexto).

Guarda cada texto en la lista `summaries`.


In [ ]:

# TODO 7.1: importa Agent, Task, Crew
# from crewai import ...

summaries = []
for art in articles:
    # TODO 7.2: extrae titular y contexto (opcional con ddg)
    # headline = ...
    # ctx = ...
    # TODO 7.3: crea el Agent (role="Investigador", goal, backstory, llm=llm)
    # researcher = ...
    # TODO 7.4: define el prompt y la Task
    # prompt = f""" ... """
    # task = Task(...)
    # TODO 7.5: ejecuta con Crew y agrega el resultado a summaries
    # result = Crew(...).kickoff()
    # text = str(getattr(result, "raw_output", result)).strip()
    # summaries.append(text)
    pass

print(f"Resúmenes creados: {len(summaries)}")



## 8) Artículo (200–250 palabras, 2–3 párrafos)
Crea un agente **Periodista** y genera el artículo usando los resúmenes.

**Pistas**:
- Une `summaries` con `\n\n` → `summaries_text`.
- Agent con `role="Periodista Deportivo"` y `llm=llm`.
- Prompt claro: “Escribe un artículo de 200–250 palabras (2–3 párrafos)...”
- Ejecuta `Crew(...).kickoff()` y guarda en `article_text`.


In [ ]:

# TODO 8.1: une resúmenes y crea Agent + Task + Crew
# summaries_text = ...
# journalist = ...
# article_prompt = f""" ... """
# article_task = ...
# article_text = str(Crew(...).kickoff())

# print("Artículo listo ✅\n")
# print(article_text)



## 9) 5 tweets (1 línea c/u, con emojis y 1–2 hashtags)
Crea un agente **Creador de Tweets** y genera EXACTAMENTE 5 líneas numeradas (1–5).

**Pistas**:
- Cada tweet debe referirse a un **dato distinto** del artículo.
- Prompt: “Crea EXACTAMENTE 5 tweets numerados (1–5)... máx 40 palabras, 1–2 hashtags, emojis.”
- Muestra las 5 líneas resultantes.


In [ ]:

# TODO 9.1: crea Agent + Task con el artículo como contexto y ejecuta
# influencer = ...
# tweets_prompt = f""" ... {article_text} ... """
# tweets_task = ...
# tweets_text = str(Crew(...).kickoff())

# TODO 9.2: imprime únicamente 5 líneas (si vienen más, recorta)
# lines = [ln.strip() for ln in tweets_text.splitlines() if ln.strip()]
# for ln in lines[:5]:
#     print(ln)
